In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, hamming_loss, precision_score, recall_score, f1_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import hamming_loss as hl

In [2]:
df = pd.read_csv('finalmovie.csv')
df.head()

,title,new_genre,corpus,genre_length,action,adventure,animation,biography,comedy,crime,...,horror,music,musical,mystery,romance,sci-fi,sport,thriller,war,western
0,Avatar: The Way of Water,"Action,Adventure,Fantasy",jake sulli life newfound famili form extrasola...,3,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Menu,"Horror,Thriller",young coupl travel remot island eat exclus res...,2,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
2,Babylon,"Comedy,Drama,History",tale outsiz ambit outrag excess trace rise fal...,3,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Everything Everywhere All at Once,"Action,Adventure,Comedy",middleag chines immigr swept insan adventur al...,3,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,M3gan,"Horror,Sci-Fi,Thriller",robot engin toy compani build lifelik doll beg...,3,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0


In [3]:
X = df['corpus']
y = df['new_genre']

In [4]:
from gensim.models import Word2Vec

# Train Word2Vec model
sentences = [doc.split() for doc in X]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Compute TF-IDF scores
tfidf_vectorizer = TfidfVectorizer()
tfidf_scores = tfidf_vectorizer.fit_transform(X)

In [6]:
# Combine Word2Vec vectors and TF-IDF scores
hybrid_vectors = []
for i, doc in enumerate(X):
    tfidf_scores_doc = tfidf_scores[i].toarray()[0]
    word_vectors_doc = []
    for word in doc.split():
        if word in w2v_model.wv.key_to_index:
            word_vector = w2v_model.wv[word]
            if word in tfidf_vectorizer.vocabulary_:
                word_vector_tfidf = word_vector * tfidf_scores_doc[tfidf_vectorizer.vocabulary_[word]]
            else:
                word_vector_tfidf = word_vector
            word_vectors_doc.append(word_vector_tfidf)
    if word_vectors_doc:
        doc_vector = np.mean(word_vectors_doc, axis=0)
    else:
        doc_vector = np.zeros(w2v_model.vector_size)
    hybrid_vectors.append(doc_vector)

### starting testing oversampling

In [ ]:
# Apply random oversampling
oversampler = RandomOverSampler(sampling_strategy='all')
X_resampled, y_resampled = oversampler.fit_resample(hybrid_vectors, y)

# Train a Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_resampled, y_resampled)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(hybrid_vectors_tsne, y, test_size=0.2, random_state=42)

In [ ]:
models = {
    # "Single Vector Machine": LinearSVC(),
    # "Logistic Regression": LogisticRegression(max_iter=10000),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
}

for model_name, model in models.items():
    # Fit the model on the resampled data
    model.fit(X_train, y_train)

    # Make predictions on test set
    y_pred = model.predict(X_test)

    # Print classification report and confusion matrix
    # print(f"Model: {model_name}")
    # print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))
    # print("Hamming Loss: {:.3f}".format(hamming_loss(y_test, y_pred)))
    # print(classification_report(y_test, y_pred, zero_division=1))
    # print(confusion_matrix(y_test, y_pred))
    print(f"Model: {model_name}")
    print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))
    print("Hamming Loss: {:.3f}".format(hl(y_test, y_pred)))
    print("F1-Score: {:.3f}".format(f1_score(y_test, y_pred, average='macro')))
    print("Precision: {:.3f}".format(precision_score(y_test, y_pred, average='macro', zero_division=1)))
    print("Recall: {:.3f}".format(recall_score(y_test, y_pred, average='macro', zero_division=1)))
    # print("Confusion Matrix:")
    # print(confusion_matrix(y_test, y_pred))
    print()

### ending oversampling test

### test

In [36]:
from sklearn.decomposition import PCA

# Create PCA object with 50 principal components
pca = PCA(n_components=50)

# Apply PCA to hybrid vectors
hybrid_vectors_pca = pca.fit_transform(hybrid_vectors)

In [38]:
from sklearn.manifold import TSNE

hybrid_vectors = np.array(hybrid_vectors)
# Create t-SNE object with 2 dimensions
tsne = TSNE(n_components=2)

# Apply t-SNE to hybrid vectors
hybrid_vectors_tsne = tsne.fit_transform(hybrid_vectors)

In [39]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(hybrid_vectors_tsne, y, test_size=0.2, random_state=42)

In [40]:
models = {
    # "Single Vector Machine": LinearSVC(),
    # "Logistic Regression": LogisticRegression(max_iter=10000),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
}

for model_name, model in models.items():
    # Fit the model on the resampled data
    model.fit(X_train, y_train)

    # Make predictions on test set
    y_pred = model.predict(X_test)

    # Print classification report and confusion matrix
    # print(f"Model: {model_name}")
    # print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))
    # print("Hamming Loss: {:.3f}".format(hamming_loss(y_test, y_pred)))
    # print(classification_report(y_test, y_pred, zero_division=1))
    # print(confusion_matrix(y_test, y_pred))
    print(f"Model: {model_name}")
    print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))
    print("Hamming Loss: {:.3f}".format(hl(y_test, y_pred)))
    print("F1-Score: {:.3f}".format(f1_score(y_test, y_pred, average='macro')))
    print("Precision: {:.3f}".format(precision_score(y_test, y_pred, average='macro', zero_division=1)))
    print("Recall: {:.3f}".format(recall_score(y_test, y_pred, average='macro', zero_division=1)))
    # print("Confusion Matrix:")
    # print(confusion_matrix(y_test, y_pred))
    print()

Model: Decision Tree
Accuracy: 0.733
Hamming Loss: 0.267
F1-Score: 0.700
Precision: 0.715
Recall: 0.721

Model: Random Forest
Accuracy: 0.738
Hamming Loss: 0.262
F1-Score: 0.717
Precision: 0.742
Recall: 0.726



### test end

# Working code

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(hybrid_vectors, y, test_size=0.2, random_state=42)

In [8]:
models = {
    "Single Vector Machine": LinearSVC(),
    "Logistic Regression": LogisticRegression(max_iter=10000),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
}

for model_name, model in models.items():
    # Fit the model on the resampled data
    model.fit(X_train, y_train)

    # Make predictions on test set
    y_pred = model.predict(X_test)

    # Print classification report and confusion matrix
    # print(f"Model: {model_name}")
    # print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))
    # print("Hamming Loss: {:.3f}".format(hamming_loss(y_test, y_pred)))
    # print(classification_report(y_test, y_pred, zero_division=1))
    # print(confusion_matrix(y_test, y_pred))
    print(f"Model: {model_name}")
    print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))
    print("Hamming Loss: {:.3f}".format(hl(y_test, y_pred)))
    print("F1-Score: {:.3f}".format(f1_score(y_test, y_pred, average='macro')))
    print("Precision: {:.3f}".format(precision_score(y_test, y_pred, average='macro', zero_division=1)))
    print("Recall: {:.3f}".format(recall_score(y_test, y_pred, average='macro', zero_division=1)))
    # print("Confusion Matrix:")
    # print(confusion_matrix(y_test, y_pred))
    print()

Model: Single Vector Machine
Accuracy: 0.179
Hamming Loss: 0.821
F1-Score: 0.032
Precision: 0.818
Recall: 0.031

Model: Logistic Regression
Accuracy: 0.180
Hamming Loss: 0.820
F1-Score: 0.012
Precision: 0.832
Recall: 0.013

Model: Decision Tree
Accuracy: 0.734
Hamming Loss: 0.266
F1-Score: 0.703
Precision: 0.732
Recall: 0.725

Model: Random Forest
Accuracy: 0.757
Hamming Loss: 0.243
F1-Score: 0.805
Precision: 0.968
Recall: 0.724



In [ ]:
# models = {
#     "Decision Tree": DecisionTreeClassifier(),
#     "Random Forest": RandomForestClassifier(),
#     "Single Vector Machine": LinearSVC()
    
# }

# metrics = ["Accuracy", "Hamming Loss", "F1-Score", "Precision", "Recall"]

# results_df = pd.DataFrame(columns=["Model"] + metrics)

# for model_name, model in models.items():
#     # Fit the model on the resampled data
#     model.fit(X_train, y_train)

#     # Make predictions on test set
#     y_pred = model.predict(X_test)

#     # Compute evaluation metrics
#     accuracy = accuracy_score(y_test, y_pred)
#     hamming_loss = hamming_loss(y_test, y_pred)
#     f1 = f1_score(y_test, y_pred, average='macro')
#     precision = precision_score(y_test, y_pred, average='macro', zero_division=1)
#     recall = recall_score(y_test, y_pred, average='macro', zero_division=1)

#     # Store results in the DataFrame
#     results_df = results_df.append({
#         "Model": model_name,
#         "Accuracy": accuracy,
#         "Hamming Loss": hamming_loss,
#         "F1-Score": f1,
#         "Precision": precision,
#         "Recall": recall
#     }, ignore_index=True)

#     # Print evaluation metrics
#     print(f"Model: {model_name}")
#     print("Accuracy: {:.3f}".format(accuracy))
#     print("Hamming Loss: {:.3f}".format(hamming_loss))
#     print("F1-Score: {:.3f}".format(f1))
#     print("Precision: {:.3f}".format(precision))
#     print("Recall: {:.3f}".format(recall))
#     # print("Confusion Matrix:")
#     # print(confusion_matrix(y_test, y_pred))
#     print()

In [10]:
from sklearn.ensemble import VotingClassifier

# Create the individual models
dt = DecisionTreeClassifier()
rf = RandomForestClassifier()

# Create the voting classifier
voting = VotingClassifier(
    estimators=[('dt', dt), ('rf', rf)],
    voting='hard'
)

# Train the voting classifier
voting.fit(X_train, y_train)

# Make predictions on test set
y_pred = voting.predict(X_test)

# Print evaluation metrics
print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))
print("Hamming Loss: {:.3f}".format(hamming_loss(y_test, y_pred)))
print("F1-Score: {:.3f}".format(f1_score(y_test, y_pred, average='macro')))
print("Precision: {:.3f}".format(precision_score(y_test, y_pred, average='macro', zero_division=1)))
print("Recall: {:.3f}".format(recall_score(y_test, y_pred, average='macro', zero_division=1)))
# print("Confusion Matrix:")
# print(confusion_matrix(y_test, y_pred))
print()

Accuracy: 0.743
Hamming Loss: 0.257
F1-Score: 0.737
Precision: 0.807
Recall: 0.723



In [18]:
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Load the pre-trained Word2Vec model and TF-IDF vectorizer
sentences = [doc.split() for doc in df['corpus']]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)
tfidf_vectorizer = TfidfVectorizer()
tfidf_scores = tfidf_vectorizer.fit_transform(df['corpus'])

# Combine Word2Vec vectors and TF-IDF scores
hybrid_vectors = []
for i, doc in enumerate(df['corpus']):
    tfidf_scores_doc = tfidf_scores[i].toarray()[0]
    word_vectors_doc = []
    for word in doc.split():
        if word in w2v_model.wv.key_to_index:
            word_vector = w2v_model.wv[word]
            if word in tfidf_vectorizer.vocabulary_:
                word_vector_tfidf = word_vector * tfidf_scores_doc[tfidf_vectorizer.vocabulary_[word]]
            else:
                word_vector_tfidf = word_vector
            word_vectors_doc.append(word_vector_tfidf)
    if word_vectors_doc:
        doc_vector = np.mean(word_vectors_doc, axis=0)
    else:
        doc_vector = np.zeros(w2v_model.vector_size)
    hybrid_vectors.append(doc_vector)

# Train the Random Forest Classifier on the entire dataset
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(hybrid_vectors, df['new_genre'])

# Take user input and predict the genre
user_input = input("Enter the synopsis of the movie: ")
user_input_processed = tfidf_vectorizer.transform([user_input])
user_input_vector = np.zeros(w2v_model.vector_size)
for word in user_input.split():
    if word in w2v_model.wv.key_to_index:
        word_vector = w2v_model.wv[word]
        if word in tfidf_vectorizer.vocabulary_:
            word_vector_tfidf = word_vector * user_input_processed[0, tfidf_vectorizer.vocabulary_[word]]
        else:
            word_vector_tfidf = word_vector
        user_input_vector += word_vector_tfidf
user_input_vector /= len(user_input.split())

genre_prediction = rfc.predict([user_input_vector])[0]

print("Predicted genre:", genre_prediction)

Enter the synopsis of the movie:  group of people travels to the middle earth


Predicted genre: Action,Adventure,Sci-Fi


In [21]:
import os
import pickle
import gzip

# Create a directory called 'new_pickle' if it does not exist
if not os.path.exists('new_pickle'):
    os.makedirs('new_pickle')

# Pickle the objects
with gzip.open("new_pickle/model.pklz", "wb") as f:
    pickle.dump(w2v_model, f)
    pickle.dump(tfidf_vectorizer, f)
    pickle.dump(rfc, f)

# Get compressed file size
file_size = os.path.getsize("new_pickle/model.pklz")
print("Compressed file size:", file_size, "bytes")

Compressed file size: 119891646 bytes


## Gradio App

In [ ]:
import gradio as gr
import pickle
import gzip
import numpy as np

# Load the pickled objects
with gzip.open("new_pickle/model.pklz", "rb") as f:
    w2v_model = pickle.load(f)
    tfidf_vectorizer = pickle.load(f)
    rfc = pickle.load(f)

def predict_genre(synopsis):
    # Process user input
    user_input_processed = tfidf_vectorizer.transform([synopsis])
    user_input_vector = np.zeros(w2v_model.vector_size)
    for word in synopsis.split():
        if word in w2v_model.wv.key_to_index:
            word_vector = w2v_model.wv[word]
            if word in tfidf_vectorizer.vocabulary_:
                word_vector_tfidf = word_vector * user_input_processed[0, tfidf_vectorizer.vocabulary_[word]]
            else:
                word_vector_tfidf = word_vector
            user_input_vector += word_vector_tfidf
    user_input_vector /= len(synopsis.split())

    # Predict the genre
    genre_prediction = rfc.predict([user_input_vector])[0]
    return genre_prediction


# Create the Gradio interface
synopsis_input = gr.Textbox(label="Enter the synopsis of the movie")
genre_output = gr.Textbox(label="Predicted genre")
gr.Interface(fn=predict_genre, inputs=synopsis_input, outputs=genre_output, title="GenreOracle",
    description="Here's a sample synopsis\n aliens in planet pandora protecting their world from human\nfamily members encounters their teenage kids in love").launch()

In [4]:
# import gradio as gr
# import pickle
# import gzip
# import numpy as np
# import pandas as pd

# # Load the pickled objects
# with gzip.open("new_pickle/model.pklz", "rb") as f:
#     w2v_model = pickle.load(f)
#     tfidf_vectorizer = pickle.load(f)
#     rfc = pickle.load(f)

# # Load the movie data from the CSV file
# movie_data = pd.read_csv("finalmovie.csv")

# def predict_genre(synopsis):
#     # Process user input
#     user_input_processed = tfidf_vectorizer.transform([synopsis])
#     user_input_vector = np.zeros(w2v_model.vector_size)
#     for word in synopsis.split():
#         if word in w2v_model.wv.key_to_index:
#             word_vector = w2v_model.wv[word]
#             if word in tfidf_vectorizer.vocabulary_:
#                 word_vector_tfidf = word_vector * user_input_processed[0, tfidf_vectorizer.vocabulary_[word]]
#             else:
#                 word_vector_tfidf = word_vector
#             user_input_vector += word_vector_tfidf
#     user_input_vector /= len(synopsis.split())

#     # Predict the genre
#     genre_prediction = rfc.predict([user_input_vector])[0]
    
#     # Filter the movie titles based on the predicted genre
#     movie_titles = movie_data[movie_data['new_genre'] == genre_prediction]['title'].tolist()
    
#     # Return the predicted genre and movie titles
#     return genre_prediction, movie_titles


# # Create the Gradio interface
# synopsis_input = gr.Textbox(label="Enter the synopsis of the movie")
# genre_output = gr.Textbox(label="Predicted Genre")
# titles_output = gr.Textbox(label="Suggested Movie Titles")
# gr.Interface(fn=predict_genre, inputs=synopsis_input, outputs=[genre_output, titles_output], title="GenreOracle",
#     description="Here's a sample synopsis\n aliens in planet pandora protecting their world from human\nfamily members encounters their teenage kids in love").launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [5]:
import gradio as gr
import pickle
import gzip
import numpy as np
import pandas as pd

# Load the pickled objects
with gzip.open("new_pickle/model.pklz", "rb") as f:
    w2v_model = pickle.load(f)
    tfidf_vectorizer = pickle.load(f)
    rfc = pickle.load(f)

# Load the movie data from the CSV file
movie_data = pd.read_csv("finalmovie.csv")

def predict_genre(synopsis):
    # Process user input
    user_input_processed = tfidf_vectorizer.transform([synopsis])
    user_input_vector = np.zeros(w2v_model.vector_size)
    for word in synopsis.split():
        if word in w2v_model.wv.key_to_index:
            word_vector = w2v_model.wv[word]
            if word in tfidf_vectorizer.vocabulary_:
                word_vector_tfidf = word_vector * user_input_processed[0, tfidf_vectorizer.vocabulary_[word]]
            else:
                word_vector_tfidf = word_vector
            user_input_vector += word_vector_tfidf
    user_input_vector /= len(synopsis.split())

    # Predict the genre
    genre_prediction = rfc.predict([user_input_vector])[0]
    
    # Filter the movie titles based on the predicted genre
    movie_titles = movie_data[movie_data['new_genre'] == genre_prediction]['title'].tolist()
    
    # Limit the number of movie titles displayed to 10
    if len(movie_titles) > 10:
        movie_titles = movie_titles[:10]
    
    # Format the movie titles as a bulleted list
    movie_titles_bullet = "\n".join([f"- {title}" for title in movie_titles])
    
    # Return the predicted genre and movie titles
    return genre_prediction, movie_titles_bullet


# Create the Gradio interface
synopsis_input = gr.Textbox(label="Enter the synopsis of the movie")
genre_output = gr.Textbox(label="Predicted genre")
titles_output = gr.Textbox(label="Movie titles with the predicted genre (up to 10)")
gr.Interface(fn=predict_genre, inputs=synopsis_input, outputs=[genre_output, titles_output], title="GenreOracle",
    description="Here's a sample synopsis\n aliens in planet pandora protecting their world from human\nfamily members encounters their teenage kids in love").launch()


Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB
Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


# Final APP

In [ ]:
import gradio as gr
import pickle
import gzip
import numpy as np
import pandas as pd


with gzip.open("new_pickle/model.pklz", "rb") as f:
    w2v_model = pickle.load(f)
    tfidf_vectorizer = pickle.load(f)
    rfc = pickle.load(f)


movie_data = pd.read_csv("finalmovie.csv")

def predict_genre(synopsis, num_titles):
    # Process user input
    user_input_processed = tfidf_vectorizer.transform([synopsis])
    user_input_vector = np.zeros(w2v_model.vector_size)
    for word in synopsis.split():
        if word in w2v_model.wv.key_to_index:
            word_vector = w2v_model.wv[word]
            if word in tfidf_vectorizer.vocabulary_:
                word_vector_tfidf = word_vector * user_input_processed[0, tfidf_vectorizer.vocabulary_[word]]
            else:
                word_vector_tfidf = word_vector
            user_input_vector += word_vector_tfidf
    user_input_vector /= len(synopsis.split())

    # Predict the genre
    genre_prediction = rfc.predict([user_input_vector])[0]
    
    # Filter the movie titles based on the predicted genre
    movie_titles = movie_data[movie_data['new_genre'] == genre_prediction]['title'].tolist()
    
    # Limit the number of movie titles displayed based on the user input
    if len(movie_titles) > num_titles:
        movie_titles = movie_titles[:num_titles]
    
    # Format the movie titles as a bulleted list
    movie_titles_bullet = "\n".join([f"- {title}" for title in movie_titles])
    
    # Return the predicted genre and movie titles
    return genre_prediction, movie_titles_bullet


# Create the Gradio interface with a Slider for the number of suggested movie titles
synopsis_input = gr.Textbox(label="Enter the synopsis of the movie")
num_titles_input = gr.Slider(minimum=5, maximum=20, step=5, label="Number of suggested movie titles")
genre_output = gr.Textbox(label="Predicted genre")
titles_output = gr.Textbox(label="Suggested movie titles with the predicted genre")
gr.Interface(fn=predict_genre, inputs=[synopsis_input, num_titles_input], outputs=[genre_output, titles_output], 
             title="GenreOracle (v1.0-beta1)",
             description="Enter the synopsis of a movie and get suggestions for similar movies").launch()